In [0]:
from pyspark.sql.functions import lit, col, when
import pandas as pd

def clean_nationality(original_df):
    # replace vague nationality with None
    exclude_list = [
        "기타", "국제연합", "미등록국가", "교황청", "무국적", "교포", "미상",
        "영국속국민","영국속령지시민","영국외지민","영국외지시민","영국해외영토시민",
        "구주기타", "중동기타", "미주기타", "대양주기타", "아프리카주기타", "아시아주기타"
    ]

    # clean up nationalities
    replacement_dict = {
        # 러시아 관련
        "러시아(연방)": "러시아",
        "러시아연방": "러시아",
        "한국계러시아인": "러시아",
        
        # 남수단
        "남수단공화국": "남수단",
        
        # 그루지야 / 조지아
        "그루지야": "조지아",
        
        # 프랑스
        "불령가이아나": "프랑스",
        "프랑스령 가이아나": "프랑스",

        # 아르메니아
        "아르메": "아르메니아",
        
        # 세르비아몬테네그로 → 세르비아  
        "세르비아몬테네그로": "세르비아",
        
        # 러시아/중국/홍콩 관련
        "홍콩거주난민": "홍콩",
        "한국계중국인": "중국",
        "타이완": "대만",
        
        # 긴 명칭 간소화
        "남아프리카공화국": "남아프리카",
        "도미니카공화국": "도미니카",
        "도미니카연방": "도미니카",
        "티모르민주공화국": "티모르",
        "예멘": "예멘공화국",
        
        # 기타 소형 섬 / 특수 영토 간소화
        "미국인근섬": "미국",
        "마르티니크": "프랑스",
        "마카오": "중국",
        "세인트빈센트그레나딘": "세인트빈센트",  
        "세인트크리스토퍼네비스": "세인트키츠네비스",
        "영령인도양섬": "영국"
        
    }
    cleaned_df = original_df.withColumn(
            "Nationality",
            when(col("Nationality").isin(exclude_list), "Undefined")
            .otherwise(col("Nationality"))
    ).replace(replacement_dict, subset=['Nationality'])
    # cleaned_df = original_df.select(
    #     [ col(c) for c in original_df.columns if c != "Nationality" ] + 
    #     [when(col("Nationality").isin(exclude_list), "Undefined").otherwise(col("Nationality")).alias("Nationality")]
        
    #     #, "Year"
    #     #, "Amount"
    # ).replace(replacement_dict, subset = ['Nationality'])
    return cleaned_df

def map_nationality(cleaned_df):
    # read csv file containing map country code
    df = pd.read_csv("/Workspace/Repos/o3oynyn@gmail.com/data-analysis_business-growth-opportunity/silver/Nationality_mapping.csv", index_col=None)

    # convert pandas df to spark df
    mapping_df = spark.createDataFrame(df)

    mapped_df = cleaned_df.join(mapping_df, \
                                cleaned_df['Nationality'] == mapping_df['Korean_Nationality'], \
                                how = "left")\
                            .select(
                                [col(c) for c in cleaned_df.columns] 
                                + [col(c) for c in mapping_df.columns if c != "Korean_Nationality"]
                            )
    return mapped_df


print('Resident UDF has been set up.')